## 데이터 파이프라인 End-to-End 프로젝트

<img src="https://velog.velcdn.com/images/newnew_daddy/post/7422a14f-8952-4990-9aba-578695cce6f7/image.png" width="60%">

1. postgresql DB에 Fake 유저 정보 저장
2. Pipeline 모듈을 통한 데이터 이행 및 데이터 가공
3. MYSQL DB에 가공된 테이블 저장
4. Streamlit 라이브러리를 통한 MYSQL 테이블 데이터 시각화
5. n초 마다 Batch성 이행을 통해 시각화 테이블 업데이트 내용 확인

### 1. PostgreSQL DB에 Fake 유저 정보 저장

##### 1) Fake 데이터 생성

Faker 라이브러리
- [공식 Docs](https://faker.readthedocs.io/en/master/)
- [Fake data 사용하기](https://www.daleseo.com/python-faker/)

In [1]:
# Fake data

from faker import Faker

fake = Faker("ko_KR")
fake.profile()

{'job': '기타 전기/전자기기 설치 및 수리원',
 'company': '유한회사 김',
 'ssn': '660512-1725653',
 'residence': '대구광역시 동구 석촌호수로 (정호이동)',
 'current_location': (Decimal('-65.3007225'), Decimal('165.333745')),
 'blood_group': 'B-',
 'website': ['http://jusighoesa.com/',
  'http://ihan.kr/',
  'https://bagno.com/',
  'http://hwango.com/'],
 'username': 'junhyeoggweon',
 'name': '허보람',
 'sex': 'F',
 'address': '제주특별자치도 청주시 청원구 석촌호수길 (성호황박면)',
 'mail': 'jeongja94@nate.com',
 'birthdate': datetime.date(1984, 3, 19)}

In [4]:
# 특정 Key의 data들만 사용

key_list = ['name', 'ssn', 'job', 'residence', 'blood_group', 'sex', 'birthdate']

data = fake.profile()

datas = dict()

for key in key_list:
    datas[key] = data[key]

datas
# {key:data[key] for key in key_list}

{'name': '이상철',
 'ssn': '530807-1610321',
 'job': '변호사',
 'residence': '울산광역시 송파구 학동길 (은서최마을)',
 'blood_group': 'A+',
 'sex': 'M',
 'birthdate': datetime.date(1977, 8, 8)}

In [ ]:
import shortuuid

data_dict["uuid"] = shortuuid.uuid()
data_dict

In [ ]:
data_dict["birthdate"].strftime("%Y%m%d")

In [7]:
# 데이터 가공
import shortuuid


key_list = ['name', 'ssn', 'job', 'residence', 'blood_group', 'sex', 'birthdate']
data = fake.profile()
user_data = dict()

for key in key_list:
    user_data[key] = data[key]
    
    if key == 'birthdate':
        user_data[key] = data[key].strftime("%Y%m%d")

user_data['uuid'] = shortuuid.uuid()

user_data

{'name': '하은서',
 'ssn': '090611-1766741',
 'job': '기타 여가 및 스포츠 관련 종사원',
 'residence': '인천광역시 성북구 양재천15가 (유진김동)',
 'blood_group': 'O-',
 'sex': 'F',
 'birthdate': '19140501',
 'uuid': 'g5gRN67Gh3r3V52GKHWtsb'}

In [23]:
## insert 함수 생성
from faker import Faker
import shortuuid

def create_fake_user():
    key_list = ['name', 'ssn', 'job', 'residence', 'blood_group', 'sex', 'birthdate']

    data = fake.profile()

    data_dict = dict()

    for key in key_list:
        
        data_dict[key] = data[key]

        if key == "birthdate": 
            data_dict[key] = data[key].strftime("%Y%m%d")
    
    data_dict['uuid'] = shortuuid.uuid()

    return data_dict


In [ ]:
create_fake_user()

In [ ]:
import pandas as pd

# datas = []

# for row in range(8):
#     datas.append(create_fake_user())

datas = [create_fake_user() for _ in range(5)]

pd.DataFrame(datas)

In [24]:
import pandas as pd

def create_fake_dataframe(n):
    data = [create_fake_user() for _ in range(n)]
    
    df = pd.DataFrame(data)
    
    return df

In [25]:
pandas_df = create_fake_dataframe(5)

pandas_df.head()

,name,job,residence,blood_group,sex,birthdate,uuid
0,임혜진,관세사,충청남도 태안군 양재천길,B+,F,1984-07-23,Bko5uWERNZSSjcW3eZnhVv
1,문성현,목제품 제조관련 종사원,부산광역시 양천구 잠실거리 (지민윤박리),O+,M,1958-09-08,QauZxRNScbcTVhVpZicWbJ
2,임은지,버스 운전원,서울특별시 서초구 영동대864가,AB+,F,2016-06-17,KYRPGXHZgJputp9S9Tigf2
3,손선영,운송관련 관리자,대전광역시 성북구 반포대0거리,O+,F,1975-10-14,RcNVvpkiNqeZDAPg4Lc7Dr
4,곽준혁,제품 디자이너,광주광역시 강동구 압구정3거리 (승현홍지면),O+,M,1937-02-02,NBf2tKB5L8fxtiigEKUci2


##### 2) Fake dataframe 저장

In [11]:
from db.connector import DBconnector
from settings import DB_SETTINGS

db_connector = DBconnector(**DB_SETTINGS['POSTGRES'])

with db_connector as connected:
    
    orm_con = connected.orm_conn
    fake_df = create_fake_dataframe(5)
    fake_df.to_sql("fake_dataframe_test", orm_con, if_exists="append", index=False)
    

접속
종료


##### 3) Fake data Insert

In [ ]:
from db.connector import DBconnector
from settings import DB_SETTINGS
from fake_user.create import create_fake_dataframe

db_connector = DBconnector(**DB_SETTINGS['POSTGRES'])

fake_df = create_fake_dataframe(10)
with db_connector as connected:
     orm_con = connected.orm_conn
     fake_df.to_sql("fake_temp", con=orm_con, if_exists='append', index=False)

##### 4) 함수 생성

In [ ]:
from db.connector import DBconnector
from settings import DB_SETTINGS
from fake_user.create import create_fake_dataframe
from random import randint

def insert_fake_dataframe():
    num = randint(5,15)
    db_connector = DBconnector(**DB_SETTINGS['POSTGRES'])

    fake_df = create_fake_dataframe(num)
    with db_connector as connected:
        orm_con = connected.orm_conn
        fake_df.to_sql("fake_temp", con=orm_con, if_exists='append', index=False)

### 2. 모듈을 통한 데이터 이행 및 데이터 가공

##### 1) 데이터 가공
- Pandas를 통해 Data Mart 형태의 테이블로 가공
  1. 거주하는 도시 통계 -> `residence` 전처리 및 컬럼 생성
  2. 혈액형 통계 -> `blood_group`	 전처리 및 컬럼 생성
  3. 남녀 통계 -> `sex` 컬럼 사용
  4. 나이대 통계 -> `birthdate` 전처리 및 컬럼 생성

In [13]:
import pandas as pd
import psycopg2

# database connection 생성
db = psycopg2.connect(host='localhost', dbname='postgres', user='hyunsoo', password='150808', port=5432)

pd.read_sql("SELECT * FROM fake_dataframe_test", db)

C:\Users\YEIN\AppData\Local\Temp\ipykernel_34360\1041671446.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT * FROM fake", db)


,name,job,residence,blood_group,sex,birthdate,uuid
0,박정숙,컴퓨터시스템 설계 및 분석가,제주특별자치도 성남시 중원구 봉은사가,A+,M,19940808,5Em6nswVuSUVPKFgkn8tTs
1,정현정,컴퓨터 강사,광주광역시 용산구 강남대709로 (영철김백면),O-,M,19331118,jcqSxMioieXtqD5PEbxUfw
2,강민준,기타 채굴 및 토목 관련 종사자,경상남도 태백시 오금거리 (지은윤리),O+,F,19450623,NCDMxqsfreLmFJQ3NB7ahZ
3,권정남,주조원,대전광역시 노원구 논현가,B-,M,19890713,fwd9ETFhq9tiXwqcDcseKA
4,심동현,위생사,광주광역시 용산구 봉은사로,A+,F,19230512,fTFcMBYzjcnMCTcsrffJVf
5,남성훈,곡식작물 재배원,대구광역시 도봉구 역삼82가,B-,M,19820204,JmJR2nmUk7Ev2bUiK3m7sf
6,강서연,한복 제조원,세종특별자치시 용산구 오금66길 (영희백윤면),B+,F,19461201,mXnrHam8eTD9GEhjXw4BXr
7,이명숙,장례 상담원 및 장례 지도사,경기도 서산시 선릉85거리 (영진김읍),O-,M,19120913,JbVapu8nfB3aeq2hdugyhK
8,신광수,단조기 조작원,대구광역시 영등포구 도산대31거리 (현정차리),B+,F,19340814,8guvhmDQ6spk8ntt9HVZmQ
9,박영식,인터넷 판매원,강원도 남양주시 가락49거리,B-,F,19240816,UatkhJB5rgeeX2UpySGZm6


In [ ]:
df = pd.read_sql("SELECT * FROM fake_dataframe_test", db)

In [35]:
# 도시 컬럼 생성 (세종특별자치시 중구 선릉8길 -> 세종특별자치시)

df['city'] = df['residenct'].str.split().str[0]
df.head()

,uuid,name,ssn,job,residenct,blood_group,sex,birthdate,city
0,WbA7aKjcFkwEaWy94xavJK,김미경,310011-1308197,경영 및 진단 전문가,광주광역시 서대문구 영동대가 (지아김이동),AB+,F,19540305,광주광역시
1,9V8oDsu6rY2xaFL2PpoFZi,이서영,050626-1443817,치과 의사,경기도 서산시 압구정71길 (정웅김리),A-,F,20180507,경기도
2,K2q362jgV3HQBZ7my2Wnj3,백정자,940208-1044850,영업 및 판매 관련 관리자,경상남도 청주시 흥덕구 도산대로,B-,F,19380724,경상남도
3,KC9YHiZxcwmNMnSZQ37U8d,장상호,850521-1526902,기타 식품가공관련 종사원,전라남도 횡성군 백제고분709가,B-,M,20110824,전라남도
4,24e3E64Ci5G63AH2TQB5KH,김지아,240403-2387302,주방 보조원,울산광역시 송파구 삼성가 (수민박리),B+,M,19971131,울산광역시


In [21]:
# 출생년도 컬럼 생성

fake_df["birthyear"] = fake_df["birthdate"].str.slice(0, 4)
fake_df.head(1)

,uuid,name,ssn,job,residenct,blood_group,sex,birthdate,city,birthyear
0,WbA7aKjcFkwEaWy94xavJK,김미경,310011-1308197,경영 및 진단 전문가,광주광역시 서대문구 영동대가 (지아김이동),AB+,F,19540305,광주광역시,1954
1,9V8oDsu6rY2xaFL2PpoFZi,이서영,050626-1443817,치과 의사,경기도 서산시 압구정71길 (정웅김리),A-,F,20180507,경기도,2018
2,K2q362jgV3HQBZ7my2Wnj3,백정자,940208-1044850,영업 및 판매 관련 관리자,경상남도 청주시 흥덕구 도산대로,B-,F,19380724,경상남도,1938
3,KC9YHiZxcwmNMnSZQ37U8d,장상호,850521-1526902,기타 식품가공관련 종사원,전라남도 횡성군 백제고분709가,B-,M,20110824,전라남도,2011
4,24e3E64Ci5G63AH2TQB5KH,김지아,240403-2387302,주방 보조원,울산광역시 송파구 삼성가 (수민박리),B+,M,19971131,울산광역시,1997


In [26]:
# 나이 컬럼 생성 (2024 - 출생년도)
current_year = 2024

fake_df["age"] = current_year - fake_df["birthyear"].astype(int)
fake_df.head(1)

,uuid,name,ssn,job,residenct,blood_group,sex,birthdate,city,birthyear,age,age_category
0,WbA7aKjcFkwEaWy94xavJK,김미경,310011-1308197,경영 및 진단 전문가,광주광역시 서대문구 영동대가 (지아김이동),AB+,F,19540305,광주광역시,1954,70,70대
1,9V8oDsu6rY2xaFL2PpoFZi,이서영,050626-1443817,치과 의사,경기도 서산시 압구정71길 (정웅김리),A-,F,20180507,경기도,2018,6,0대
2,K2q362jgV3HQBZ7my2Wnj3,백정자,940208-1044850,영업 및 판매 관련 관리자,경상남도 청주시 흥덕구 도산대로,B-,F,19380724,경상남도,1938,86,80대
3,KC9YHiZxcwmNMnSZQ37U8d,장상호,850521-1526902,기타 식품가공관련 종사원,전라남도 횡성군 백제고분709가,B-,M,20110824,전라남도,2011,13,10대
4,24e3E64Ci5G63AH2TQB5KH,김지아,240403-2387302,주방 보조원,울산광역시 송파구 삼성가 (수민박리),B+,M,19971131,울산광역시,1997,27,20대


In [31]:
# 혈액형 컬럼 생성 (AB+ -> AB)

fake_df["blood"] = fake_df["blood_group"].str.slice(0, -1)
fake_df.head(1)

,uuid,name,ssn,job,residenct,blood_group,sex,birthdate,city,birthyear,age,age_category,blood
0,WbA7aKjcFkwEaWy94xavJK,김미경,310011-1308197,경영 및 진단 전문가,광주광역시 서대문구 영동대가 (지아김이동),AB+,F,19540305,광주광역시,1954,70,70대,AB
1,9V8oDsu6rY2xaFL2PpoFZi,이서영,050626-1443817,치과 의사,경기도 서산시 압구정71길 (정웅김리),A-,F,20180507,경기도,2018,6,0대,A
2,K2q362jgV3HQBZ7my2Wnj3,백정자,940208-1044850,영업 및 판매 관련 관리자,경상남도 청주시 흥덕구 도산대로,B-,F,19380724,경상남도,1938,86,80대,B
3,KC9YHiZxcwmNMnSZQ37U8d,장상호,850521-1526902,기타 식품가공관련 종사원,전라남도 횡성군 백제고분709가,B-,M,20110824,전라남도,2011,13,10대,B
4,24e3E64Ci5G63AH2TQB5KH,김지아,240403-2387302,주방 보조원,울산광역시 송파구 삼성가 (수민박리),B+,M,19971131,울산광역시,1997,27,20대,B


In [ ]:
# 나이대 컬럼 생성

def categorize_age(age):

    if age >= 100:
        return "90대 이상"
    else:
        return str(age // 10 * 10) + "대"

fake_df["age_category"] = fake_df["age"].apply(categorize_age)
fake_df.head(1)

In [33]:
# 컬럼 정렬

fake_df[["uuid", "name", 'job', "sex", "city", "birthyear", "age", "blood", "age_category"]]

,uuid,name,job,sex,city,birthyear,age,age_category,blood
0,WbA7aKjcFkwEaWy94xavJK,김미경,경영 및 진단 전문가,F,광주광역시,1954,70,70대,AB
1,9V8oDsu6rY2xaFL2PpoFZi,이서영,치과 의사,F,경기도,2018,6,0대,A
2,K2q362jgV3HQBZ7my2Wnj3,백정자,영업 및 판매 관련 관리자,F,경상남도,1938,86,80대,B
3,KC9YHiZxcwmNMnSZQ37U8d,장상호,기타 식품가공관련 종사원,M,전라남도,2011,13,10대,B
4,24e3E64Ci5G63AH2TQB5KH,김지아,주방 보조원,M,울산광역시,1997,27,20대,B


##### 2) 데이터 가공 함수 생성

In [ ]:
def transform_data(pandas_df):
    current_year = 2024

    # 도시 이름 컬럼 생성
    pandas_df["city"] = pandas_df["residence"].str.split().str[0]

    # 출생년도 컬럼 생성
    pandas_df["birthyear"] = pandas_df["birthdate"].str.slice(0, 4)

    # 혈액형 컬럼 생성
    pandas_df["blood"] = pandas_df["blood_group"].str.slice(0, -1)

    # 나이 컬럼 생성
    pandas_df["age"] = current_year - pandas_df["birthyear"].astype(int)

    # 나이대 컬럼 생성
    pandas_df["age_category"] = pandas_df["age"].apply(categorize_age)

    df = fake_df[["uuid", "name", 'job', "sex", "city", "birthyear", "age", "blood", "age_category"]]

    return df

In [ ]:
fake_df = create_fake_dataframe(10)

transform_data(fake_df)

##### 3) 데이터 이행 모듈에 통합

In [1]:
from db.connector import DBconnector
from db.postgresql_query import queries
from settings import DB_SETTINGS, TEMP_PATH
from pipeline.extract import extractor
from pipeline.data_transform import transformer
from pipeline.load import loader

In [ ]:
batch_date = "20240202"

def main(batch_date):
    
    db_connector = DBconnector(**DB_SETTINGS['POSTGRES'])
    
    for table_name in queries:
        print(table_name)
        pandas_df = extractor(db_connector, table_name, batch_date)
        
        res, df = transformer(batch_date, table_name, pandas_df)
        
        db_connector = DBconnector(**DB_SETTINGS['MYSQL'])
        
        loader(db_connector, df, table_name)
            
main(batch_date)
    

### 3. start.py에 반영
- 특정 간격(3초, 5초 등) 기준으로 fake data가 Insert 될 수 있도록 설정

In [ ]:
from datetime import datetime, timedelta, timezone
import click, time
from pipeline import controller

from fake_data.insert import insert_data

@click.command
@click.option('-d', '--custom-batch-date', type=click.STRING, default='', help='배치작업연월일')

def start_batch(custom_batch_date):
    
    for _ in range(10):
        print("START BATCH")
        insert_data()
        
        if custom_batch_date:
            batch_date = custom_batch_date
        else:
            batch_date = (datetime.now(timezone(timedelta(hours=9))) - timedelta(days = 1)).strftime('%Y%m%d')
            
        controller.main(batch_date)
        time.sleep(5)
        
if __name__ == "__main__":
    
    start_batch()

### 4. 시각화 코드 작성 (streamlit)

- requirements.txt 파일에 라이브러리 추가

  ```
  shortuuid==1.0.11
  streamlit==1.31.1
  streamlit-autorefresh==1.0.1
  altair==4.0
  vega_datasets==0.9.0
  ```
  
> 웹페이지 시작 : streamlit run [시각화 Python 파일]

In [ ]:
import streamlit as st
import pandas as pd

from db.connector import DBconnector
from settings import DB_SETTINGS

## MYSQL DB에서 DataMart 데이터 읽기
db_connector = DBconnector(**DB_SETTINGS['MYSQL'])
with db_connector as connected:
    
    df = pd.read_sql("SELECT * FROM fake_datamart;", connected.conn)

## 차트에 사용할 데이터 생성
df_sex = df.groupby('sex').size().reset_index(name='count')
df_blood = df.groupby('blood').size().reset_index(name='count')
df_city = df.groupby('city').size().reset_index(name='count')
df_age = df.groupby('age_category').size().reset_index(name='count')

## 차트 그리기
# Row Count
st.title(f'TOTAL DATA : {len(df)}')
st.dataframe(df.head())

# 성별
st.header('SEX GROUP BY')
st.bar_chart(df_sex, x='sex', y='count')

# 혈액형
st.header('BLOOD TYPE GROUP BY')
st.bar_chart(df_blood, x='blood', y='count')

# 거주 도시
st.header('CITY GROUP BY')
st.bar_chart(df_city, x='city', y='count')

# 나이
st.header('AGE GROUP BY')
st.line_chart(df_age, x='age_category', y='count')